# FUNDUS - The urban geography of inequalities
![Urbanum logo](imgs/urbanum_2.png)
## Data used
### WEkEO datasets
Dataset queries were generated by using the [WEkEO online platform](https://www.wekeo.eu/data).
The queries can be fund in the `data/jsons` folder
+ Global 10-daily Leaf Area Index 333m
```json
{
  "datasetId": "EO:CLMS:DAT:CGLS_GLOBAL_LAI300_V1_333M",
  "dateRangeSelectValues": [
    {
      "name": "dtrange",
      "start": "2022-06-01T00:00:00.000Z",
      "end": "2022-06-30T23:59:59.999Z"
    }
  ]
}
```
+ Level 2 Land - Sea and Land Surface
Temperature Radiometer (SLSTR) - Sentinel-3
```json
{
  "datasetId": "EO:ESA:DAT:SENTINEL-3:SL_2_LST___",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        18.99804053609134,
        47.42120186691113,
        19.190237776905892,
        47.58048586099437
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2022-06-01T00:00:00.000Z",
      "end": "2022-06-30T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "productType",
      "value": "LST"
    },
    {
      "name": "timeliness",
      "value": "Near+Real+Time"
    },
    {
      "name": "orbitDirection",
      "value": "ascending"
    },
    {
      "name": "processingLevel",
      "value": "LEVEL2"
    }
  ]
}
```
+ Global 10-daily Fraction of Vegetation Cover 333m
```json
{
  "datasetId": "EO:CLMS:DAT:CGLS_GLOBAL_FCOVER300_V1_333M",
  "dateRangeSelectValues": [
    {
      "name": "dtrange",
      "start": "2022-06-01T00:00:00.000Z",
      "end": "2022-06-30T23:59:59.999Z"
    }
  ]
}
```
### External dataset
The square meter prices dataset was collected on 12th July 2022 using our [freely available scraper](https://github.com/Urbanum-Lab/budapest_property_price_scraper). The data is in the `data/aggragated` folder. **WARNING**: Check the README.md file of the scraper to get your own data.
## Outline
At the end of this notebook you will know:
+ How to aggregate data using the h3 library
+ How to visualize the data using the pydeck package
+ How to investigate spatial discrepancies in property prices and the environment

We would like to test the common conception that wealthier neighborhoods are greener and also
enjoy a lower average temperature during summer. Since we have no data on wealth at this
granularity, we use property square meter prices as a proxy of wealth. This is a
strong and untested assumption. This notebook will test the followings:
+ temperature and greenness are connected
+ temperature and square meter prices are connected
+ greenness and square meter prices are connected
We shall present our finding as simple interactive visualizations. The present work is just the beginning of a more detailed investigation, so it should be treated as an exploratory data analysis.

## Library imports
Here, we import packages for the project.

In [14]:
import json
import os
from functools import reduce

import h3
import altair as at
import numpy as np
import pandas as pd
import pydeck as pdk
import xarray as xr
from hda import Client

## Introduction
## Gathering the data
**WARNING**: Downloading the datasets takes time! The data will be downloaded into the current working directory. Use your favorite tools to move the downloaded files to the appropriate folders.
### SLSTR

In [6]:
c = Client(debug=True)
with open("../data/jsons/temperature.json") as infile:
    query = json.load(infile)
matches = c.search(query)
# matches.download()

2022-07-18 10:27:37,075 INFO Download rate 2.2M/s   
2022-07-18 10:27:37,076 DEBUG {'downloadUri': None, 'filename': 'S3B_SL_2_LST____20220628T194550_20220628T194850_20220628T221819_0180_067_299_0720_PS2_O_NR_004.SEN3.zip', 'order': None, 'productInfo': {'datasetId': 'EO:ESA:DAT:SENTINEL-3:SL_2_LST___', 'product': 'S3B_SL_2_LST____20220628T194550_20220628T194850_20220628T221819_0180_067_299_0720_PS2_O_NR_004.SEN3', 'productEndDate': '2022-06-28T19:48:49.595000Z', 'productStartDate': '2022-06-28T19:45:49.595000Z'}, 'size': 72987415, 'url': '9cc5a076-8c00-5161-9c90-a0e6d384b2f5/S3B_SL_2_LST____20220628T194550_20220628T194850_20220628T221819_0180_067_299_0720_PS2_O_NR_004.SEN3.zip'}
2022-07-18 10:27:37,076 DEBUG ===> POST https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder
2022-07-18 10:27:37,077 DEBUG ===> POST {"jobId": "d2CLuZPhvqF1CTkRMfuwrbFaSZs", "uri": "9cc5a076-8c00-5161-9c90-a0e6...
2022-07-18 10:27:52,822 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443

HTTPError: 403 Client Error: FORBIDDEN for url: https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder/status/Xz0urIDhJf66dxlyiJLh6JkvIlg

### LAI

In [ ]:
c = Client(debug=True)
with open("../data/jsons/lai.json") as infile:
    query = json.load(infile)
matches = c.search(query)
# matches.download()

### FCOVER

In [ ]:
c = Client(debug=True)
with open("../data/jsons/fcover.json") as infile:
    query = json.load(infile)
matches = c.search(query)
# matches.download()

*Your operating system and tools might be different, below you can read tips which might be useful
on a Linux machine*
+ The notebook assumes that all data is in the `data` folder.
+ Move all zip and nc files
to the folder (e.g. you can use the `mv` command, like `mv ../data "*.zip"`)
+ Make folders for the data fiels (`cd data; mkdir leaf_data temp_data fcover`)
+ Move the *.nc files to the corresponding folders (e.g. move Leaf Area Index data into `leaf_data`)
+ Move the *.zip files into the `temp_data` folder `cd temp_data; unzip "*.zip"; rm "*.zip"`

## Cleaning and transforming data
### Property square meter prices
We scraped a Hungarian real estate listing site to get property prices in Budapest. The listing entries
were geocoded using the `geocoder` package. The geo-coordinates were indexed using the [H3 hexagonal
geospatial indexing system](https://h3geo.org/). You can check the resolution table of the cell
areas [here](https://h3geo.org/docs/core-library/restable/). For more details, you can check
[the repository of the scraper](https://github.com/Urbanum-Lab/budapest_property_price_scraper).

The data looks like this:

In [15]:
df6 = pd.read_csv("../data/aggregated/l6.tsv", sep="\t")
df7 = pd.read_csv("../data/aggregated/l7.tsv", sep="\t")
df8 = pd.read_csv("../data/aggregated/l8.tsv", sep="\t")

df7.head()

,l7,price
0,871e020caffffff,1.219298e+06
1,871e02684ffffff,1.200000e+06
2,871e030a9ffffff,1.508301e+06
3,871e03134ffffff,9.494950e+05
4,871e03449ffffff,1.197017e+06


The hexagons listed in these files constitues our area of interest.
### Temperature data
The code below aggregates the average temperature data on various levels of H3 hashing and writes the results to a tsv file.

In [16]:
h3_l6 = set(df6["l6"])
h3_l7 = set(df7["l7"])
h3_l8 = set(df8["l8"])

root_folder = "../data/temp_data"
dirs = [
    os.path.join(root_folder, d)
    for d in os.listdir(root_folder)
    if os.path.isdir(os.path.join(root_folder, d))
]


def is_within_bounding_box(lat, long):
    if 47.392134 < lat < 47.601216 and 18.936234 < long < 19.250031:
        return True
    else:
        return False


latlong_temp = {}
for inpath in dirs:
    # geodetic_tx.nc -> latitude_tx, longitude_tx
    geodetic = xr.open_dataset(
        filename_or_obj=os.path.join(inpath, "geodetic_tx.nc"), engine="netcdf4"
    )
    lat = geodetic.data_vars["latitude_tx"].to_numpy().flatten()
    long = geodetic.data_vars["longitude_tx"].to_numpy().flatten()
    # met_tx.nc -> temperature_tx
    met_tx = xr.open_dataset(
        filename_or_obj=os.path.join(inpath, "met_tx.nc"), engine="netcdf4"
    )
    temp = met_tx.data_vars["temperature_tx"].to_numpy().flatten()
    # LST_ancillary_ds.nc -> NDVI (empyt :()
    lst = xr.open_dataset(
        filename_or_obj=os.path.join(inpath, "LST_ancillary_ds.nc"), engine="netcdf4"
    )
    ndvi = lst.data_vars["NDVI"].to_numpy().flatten()

    temp_data = zip(lat, long, temp)
    temp_data = (e for e in temp_data if is_within_bounding_box(e[0], e[1]))
    for e in temp_data:
        k = (e[0], e[1])
        if latlong_temp.get(k, False):
            latlong_temp[k] = (latlong_temp[k] + e[2]) / 2
        else:
            latlong_temp[k] = e[2]

with open("../data/temp_budapest.tsv", "w") as outfile:
    h = "lat\tlong\tcelsius\tl6\tl7\tl8\n"
    outfile.write(h)
    for k, v in latlong_temp.items():
        l6 = h3.geo_to_h3(k[0], k[1], 6)
        l7 = h3.geo_to_h3(k[0], k[1], 7)
        l8 = h3.geo_to_h3(k[0], k[1], 8)
        if l6 in h3_l6 and l7 in h3_l7 and l8 in h3_l8:
            o = (
                str(k[0])
                + "\t"
                + str(k[1])
                + "\t"
                + str(v - 273.15)
                + "\t"
                + l6
                + "\t"
                + l7
                + "\t"
                + l8
                + "\n"
            )
            outfile.write(o)

### Global 10-daily Leaf Area Index 333m
The code below computes the average LAI and assigns H3 hash codes to the values. The results will be saved into a tsv file.

In [18]:
root_folder = "../data/leaf_data"
fs = [
    os.path.join(root_folder, f)
    for f in os.listdir(root_folder)
    if os.path.isfile(os.path.join(root_folder, f))
]

ll2lai = {}

for f in fs:
    try:
        ds = xr.open_dataset(filename_or_obj=os.path.join(f), engine="netcdf4")
        lat = ds.data_vars["LAI"]["lat"].to_numpy()
        lat = [e for e in lat if 47.392134 < e < 47.601216]
        lon = ds.data_vars["LAI"]["lon"].to_numpy()
        lon = [e for e in lon if 18.936234 < e < 19.250031]
        time = ds.data_vars["LAI"]["time"].to_numpy()[0]
        for i in range(len(lat)):
            for j in range(len(lon)):
                one_point = ds["LAI"].sel(lat=lat[i], lon=lon[i])
                vals = one_point.values[0]
                if ll2lai.get((lat[i], lon[j]), False):
                    ll2lai[(lat[i], lon[j])] = (ll2lai[(lat[i], lon[j])] + vals) / 2.0
                else:
                    ll2lai[(lat[i], lon[j])] = vals
    except Exception as exc1:
        print(exc1)
        continue

with open("../data/lai_budapest.tsv", "w") as outfile:
    h = "lat\tlong\tlai\tl6\tl7\tl8\n"
    outfile.write(h)
    for k, v in ll2lai.items():
        h6 = h3.geo_to_h3(k[0], k[1], 6)
        h7 = h3.geo_to_h3(k[0], k[1], 7)
        h8 = h3.geo_to_h3(k[0], k[1], 8)
        if h6 in h3_l6 and h7 in h3_l7 and h8 in h3_l8:
            o = (
                str(k[0])
                + "\t"
                + str(k[1])
                + "\t"
                + str(v)
                + "\t"
                + str(h6)
                + "\t"
                + str(h7)
                + "\t"
                + str(h8)
                + "\n"
            )
            outfile.write(o)

### Global 10-daily Fraction of Vegetation Cover 333m
The code below computes the average FCOVER and assigns H3 hash codes to the values. The results will be saved into a tsv file.

In [19]:
root_folder = "../data/fcover"
fs = [
    os.path.join(root_folder, f)
    for f in os.listdir(root_folder)
    if os.path.isfile(os.path.join(root_folder, f))
]


def is_within_bounding_box(lat, long):
    if 47.392134 < lat < 47.601216 and 18.936234 < long < 19.250031:
        return True
    else:
        return False


ll2fcover = {}

for f in fs:
    try:
        ds = xr.open_dataset(filename_or_obj=os.path.join(f), engine="netcdf4")
        lat = ds.data_vars["FCOVER"]["lat"].to_numpy()
        lat = [e for e in lat if 47.392134 < e < 47.601216]
        lon = ds.data_vars["FCOVER"]["lon"].to_numpy()
        lon = [e for e in lon if 18.936234 < e < 19.250031]
        time = ds.data_vars["FCOVER"]["time"].to_numpy()[0]
        for i in range(len(lat)):
            for j in range(len(lon)):
                one_point = ds["FCOVER"].sel(lat=lat[i], lon=lon[i])
                vals = one_point.values[0]
                if ll2fcover.get((lat[i], lon[j]), False):
                    ll2fcover[(lat[i], lon[j])] = (
                        ll2fcover[(lat[i], lon[j])] + vals
                    ) / 2.0
                else:
                    ll2fcover[(lat[i], lon[j])] = vals
    except Exception as exc1:
        print(exc1)
        continue

with open("../data/fcover_budapest.tsv", "w") as outfile:
    h = "lat\tlong\tfcover\tl6\tl7\tl8\n"
    outfile.write(h)
    for k, v in ll2fcover.items():
        h6 = h3.geo_to_h3(k[0], k[1], 6)
        h7 = h3.geo_to_h3(k[0], k[1], 7)
        h8 = h3.geo_to_h3(k[0], k[1], 8)
        if h6 in h3_l6 and h7 in h3_l7 and h8 in h3_l8:
            o = (
                str(k[0])
                + "\t"
                + str(k[1])
                + "\t"
                + str(v)
                + "\t"
                + str(h6)
                + "\t"
                + str(h7)
                + "\t"
                + str(h8)
                + "\n"
            )
            outfile.write(o)

## Visualizing the data
### Maps
#### Square meter prices

In [27]:
df_price = pd.read_csv("../data/aggregated/l7.tsv", sep="\t")
df_price["normalized"] = 255 - (df_price["price"] / np.sqrt(np.sum(df_price["price"] ** 2)) * 1000)
layer = pdk.Layer(
    "H3HexagonLayer",
    df_price,
    get_hexagon="l7",
    auto_highlight=True,
    # elevation_scale=10,
    pickable=True,
    # elevation_range=[min(df["price"]), max(df["price"])],
    extruded=True,
    coverage=0.8,
    opacity=0.01,
    get_fill_color="[255, normalized, 0]",
)

view_state = pdk.ViewState(
    latitude=47.500000, longitude=19.040236, zoom=10.5, bearing=0, pitch=35
)
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "square meter price: {price}"},
)
r.to_html("../vizs/maps/prices_h7.html")

#### Temperature

In [29]:
df = pd.read_csv("../data/temp_budapest.tsv", sep="\t")
df.fillna(0, inplace=True)

df_temp = df.groupby("l7").mean()
df_temp.reset_index(inplace=True, level=["l7"])
df_temp["rescaled"] = [255 - ((e**3)/100) for e in df_temp["celsius"]]
layer = pdk.Layer(
    "H3HexagonLayer",
    df_temp,
    get_hexagon="l7",
    auto_highlight=True,
    pickable=True,
    extruded=True,
    coverage=0.8,
    opacity=0.05,
    get_fill_color="[255, rescaled, 0]",
)

view_state = pdk.ViewState(
    latitude=47.500000, longitude=19.040236, zoom=10.5, bearing=0, pitch=35
)
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "temperature (celsius): {celsius}"},
)
r.to_html("../vizs/maps/temperature_h7.html")

#### Leaf Area Index

In [30]:
df = pd.read_csv("../data/lai_budapest.tsv", sep="\t")
df.fillna(0, inplace=True)

df_lai = df.groupby("l7").mean()
df_lai.reset_index(inplace=True, level=["l7"])
layer = pdk.Layer(
    "H3HexagonLayer",
    df_lai,
    get_hexagon="l7",
    auto_highlight=True,
    pickable=True,
    extruded=True,
    coverage=0.9,
    opacity=0.05,
    get_fill_color="[255, 255 - (lai * 100), 0]"
)

view_state = pdk.ViewState(
    latitude=47.500000, longitude=19.040236, zoom=10.5, bearing=0, pitch=35
)
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "Leaf Area Index: {lai}"},
)
r.to_html("../vizs/maps/lai_h7.html")

#### Fraction of Vegetation Cover

In [31]:
df = pd.read_csv("../data/fcover_budapest.tsv", sep="\t")
df.fillna(0.0, inplace=True)

df_fcover = df.groupby("l7").mean()
df_fcover.reset_index(inplace=True, level=["l7"])
df_fcover["normalized"] = (df_fcover["fcover"] / np.sqrt(np.sum(df_fcover["fcover"] ** 5))) ** -2
df_fcover["normalized"][df_fcover["normalized"] == np.inf] = 255
layer = pdk.Layer(
    "H3HexagonLayer",
    df_fcover,
    get_hexagon="l7",
    auto_highlight=True,
    pickable=True,
    extruded=True,
    coverage=0.8,
    opacity=0.05,
    get_fill_color="[255, normalized, 0]",
)

view_state = pdk.ViewState(
    latitude=47.500000, longitude=19.040236, zoom=10.5, bearing=0, pitch=35
)
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "Fraction of Vegetation Cover: {fcover}"},
)
r.to_html("../vizs/maps/fcover_h7.html")

/tmp/ipykernel_83587/3755050406.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fcover["normalized"][df_fcover["normalized"] == np.inf] = 255


### Correlation matrix

In [24]:
data_frames = [df_price, df_temp, df_lai, df_fcover]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['l7'],
                                            how='outer'), data_frames)
df_merged.dropna(inplace=True)
df_merged.drop(columns=["normalized_x", "lat_x", "long_x", "rescaled", "lat_y",
                "long_y", "lat", "long", "normalized_y"], inplace=True)
df_merged.head()

,l7,price,celsius,lai,fcover
4,871e03449ffffff,1.197017e+06,19.883478,4.093056,0.736756
5,871e0344affffff,9.605164e+05,22.373126,3.321111,0.779938
6,871e0344bffffff,1.556710e+06,18.937453,3.557240,0.784969
7,871e03459ffffff,1.115068e+06,20.933828,3.778786,0.813342
9,871e0345dffffff,1.449016e+06,21.627415,3.191146,0.775750


In [25]:
cor_data = (df_merged.corr().stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'}))
cor_data['correlation_label'] = cor_data['correlation'].map('{:.2f}'.format)  # Round to 2 decimal
cor_data.head()

,variable,variable2,correlation,correlation_label
0,price,price,1.000000,1.00
1,price,celsius,0.082898,0.08
2,price,lai,0.130627,0.13
3,price,fcover,0.085454,0.09
4,celsius,price,0.082898,0.08


In [32]:
base = at.Chart(cor_data).encode(
    x='variable2:O',
    y='variable:O'
)

text = base.mark_text().encode(
    text='correlation_label',
    color=at.condition(
        at.datum.correlation > 0.5,
        at.value('white'),
        at.value('black')
    )
)

cor_plot = base.mark_rect().encode(
    color='correlation:Q'
).properties(
    width=700,
    height=700
)

cor_plot + text

alt.LayerChart(...)

## Further directions
Here we checked if there is a connection between house prices and the environment. We used WEkEO data to get information on the environment and we scraped property prices to supplement the data. We found only weak connection between various datasets.

In the future, we would like to increase the time window to get more LAI and FCOVER data. We investigate the possibility to incorporate other datasets like OSM.